### Développement d’un pipeline de calcul du TMB

#### Projet 4BiM, 2021

#### Auteurs : Marie Casimir, Loup Petitjean et Nicolas Mendiboure
#### Encadrantes Innate-Pharma : Sabrina Carpentier et Luciana Bastista
#### Encadrante INSA : Maïwenn Pineau

### Installation et import des modules

In [1]:
import io
import copy
import pandas as pd
import numpy as np
from collections import Counter

## Importation de vcf sample1 :

In [2]:
def check_extension(path):
    """
    Argument :
        
        path : string, chemin vers le fichier à vérifier.
        
        Cette fonction permet de bien vérifier que le fichier d'entré contient
    bien l'extension .vcf.
    
    Attention : sur Windows cela peut poser un problème car les extensions de fichiers 
    ne sont pas toujours apparentes dans les chemins.
    
    Return:
    
        True ou False si le fichier est dans les normes.
    """
    
    split = path.split(".")
    format_name = len(split) -1 

    if (split[format_name].lower() != "vcf"):
        print("Votre fichier n'est pas au bon format, format attendu : vcf")
        return(False)
    else :
        print("Succès : extension vcf détectée")
        return(True)

In [3]:
def check_format(path):
    """
    Argument :
    
        path : string, chemin vers le fichier à vérifier.
        
        Dans tous le fichiers VCF, la première informative doit être de la forme : ##format=VCFv4.x.
    Cette fonction permet de le vérifier, elle renvoie True dans le cas échéant et False sinon.
    
    Return :
        
        True ou False si le fichier est dans les normes.
    """
    with open(path, 'r') as f:
        line = f.readline()
    
    if (line.find("VCF") != -1): #Si on trouve 'VCF' dans line 
        return (True)
    else:
        return(False)

In [4]:
def check_missing_data(df):
    """
    Argument : 
    
        df : dataframe.
    
        Cette fonction permet de vérifier si le fichier vcf n'est pas érroné.
    Pour cela on regarde qu'il ne manque pas de colonne, s'il en manque, la fonction 
    renvoie False avec le nom de la colonne manquante.
    On vérifie ensuite qu'il ne manque pas d'information sur chaque variant  (ou ligne),
    pour cela on regarde qu'il n'y ait pas de NaN. S'il y a trop de NaN, elle return False,
    si le nombre de NaN est faible comparé à la taille du fichier, on supprime les lignes où
    sont localisés les NaN.
    
    Return:
    
        -1 :  le fichier n'est pas bon on le rejette ;
        
        0 : Le fichier est à la limite de l'acceptable (quelques NaN) 
        et peut subir une modification pour traiter les NaN ;
        
        1 : Le fichier est validé.
    """
    
    #Vérifier  qu'il ne manque pas une colonne dans le ficher :
    columns = ['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT']
    for col in columns:
        if (col not in df.columns):
            print("Ce fichier vcf est incomplet")
            print("Il manque la colonne {}".format(col))
            return (-1)
        
    #Vérifier que chaque ligne est bien complète (pas de NaN): 
    NaN_col = df.isna().sum(axis=0)
    NaN_cutoff = 3 #nombre de NaN admissibles par fichier vcf, au dèla fichier rejetté.
    
    if (sum(NaN_col) !=0) : #s'il y a des NaN
        if (sum(NaN_col) <= NaN_cutoff) :
            return(0)
            
        else:
            print("Votre fichier un nombre de données manquantes trop élevé.")
            print("Veuillez fournir un vcf de meilleur qualité.")
            return (-1)
    else:
        print("Contrôle des NaN : Ok")
        return(1)

In [5]:
def drop_NaN_rows(df):
    
    """
    Argument: 
    
        df : dataframe.
        
        Cette fonctionne permet de supprimer les lignes dans lesquelles on aurait des NaN.
    
    Return :
    
        new_df : dataframe dont les lignes contenant des 'NaN' ont été supprimées.
    """
    
    new_df = copy.deepcopy(df) # Pour ne pas ecraser notre df initial 
    NaN_line = df.isna().sum(axis=1)
    indexes = []
    for i, line in enumerate(NaN_line.values):
                if (line != 0):
                    indexes.append(i)
    for idx in indexes :
                new_df = df.drop([idx], axis = 0)
            
    print("Suppression des NaN : Ok")      
    return (new_df)

In [6]:
def quality_control(df):
    """
    Argument:
    
        df : dataframe à controler.
        
        Cette fonction fait appel à notre fonction check_missing_data,
    elle fait une synthèse sur la qualité de notre fichier au moment de l'importer
    dans la fonction read_vcf.
    
    Return : 
        
        df2 : dataframe avec contrôle qualité effectué.
    """
    
    miss = check_missing_data(df)
    #print(miss)
    
    if (miss == -1):
        return (False)
    
    elif (miss == 1):
        return (df)
    
    elif (miss == 0):
        df2 = drop_NaN_rows(df) #remove NaN rows
        df2.index = range(0, len(df2),1) #car les indexes ne sont plus bons à cause du df.drop
        return (df2)

In [7]:
def read_vcf(path, QC = True):
    """
    Arguments :
        
        path : string,  chemin vers le fichier vcf à importer ;
        
        QC : booléen, effectue un controle qualité si True.
    
        Cette fonction permet d'importer un fichier vcf et de le convertir en dataframe.
    Il y a la possiblité de faire en amont un contrôle qualité via l'appel de la fonction quality_control 
    si QC == True, sinon importation classique. 
    
    Return :
        
        vcf_df : dataframe du fichier vcf importé.
    """
    
    if (check_extension(path) == True) and (check_format(path) == True) :
        with open(path, 'r') as f:
            lines = [l for l in f if not l.startswith('##')]
        
        vcf_df = pd.read_csv( io.StringIO(''.join(lines)), dtype={'#CHROM': str, 'POS': int, 'ID': str, 
                                                               'REF': str, 'ALT': str,'QUAL': str, 
                                                               'FILTER': str, 'INFO': str, 'FORMAT': str}, 
                         sep='\t').rename(columns={'#CHROM': 'CHROM'})
        
        if (QC == True):
            vcf_df2 = quality_control(vcf_df) #On passe notre vcf en contrôle qualité
            print("Contrôle qualité du VCF : Ok")
            return(vcf_df2)
        
        else:
            print("Contrôle qualité du VCF : Non fait")
            return(vcf_df)

## Contrôle des filtres :

In [8]:
def quality_filter_normal(df_normal, reject = ['LowQual', 'INDEL_SPECIFIC_FILTERS;LowQual'], index = True):
    """
    Arguments : 
        
        df_normal : dataframe issu d'un vcf de tissu sain ;
        
        reject : liste des filtres à ne pas garder ;
        
        index : booleen qui indique le revoie ou non d'un indexe.
    
        Cette fonction permet de filtrer (supprimer les lignes) des variants d'un dataframe issus d'un tissu normal 
    dont le 'FILTER' est contenu dans la liste 'reject'. 
    Chaque ligne supprimée se retrouve indexée et par son numero de chromosome et par la position du variant 
    dans un tuple afin de les supprimer également en aval dans le dataframe tumoral.
    
    Return : 
    
        df_n : le dataframe normal filtré ;
        
        indexes : liste de tuples contenant les #chrom et les positions des variants supprimés 
            sur le dataframe normal pour appliquer la même opération sur le dataframe tumoral.
    """
    
    df_n = copy.deepcopy(df_normal) #deep copy pour ne pas ecraser l'original
    indexes = []

    for muta in df_normal.index:
        if df_normal["FILTER"][muta] in reject:
            
            #On stocke le #CHROM et la #POS correspondant dans un tuple,
            #pour effectuer la même suppression dans notre fichier tumoral apres :
            indexes.append( (df_normal["CHROM"][muta], df_normal["POS"][muta]))
            
            #On supprime ensuite la ligne correspondante car mauvaise qualite
            df_n = df_n.drop(labels = muta, axis=0)
            
    df_n.index = range(0, len(df_n), 1) #reajustement des indexes apres le drop
    
    if (index == True):
        return(df_n, indexes)
    else:
        return(df_n)

In [190]:
def quality_filter_tumor(df_tumor, index_normal, reject = ['LowQual', 'INDEL_SPECIFIC_FILTERS;LowQual']):
    """
    Arguments:
    
        df_tumor : dataframe issu d'un vcf de tissu tumoral ;
        
        index_normal : liste de tuples contenant les #chrom et les positions des variants supprimés
                        pour appliquer la même opération sur le dataframe tumoral;
        
        reject : liste des filtres à ne pas garder ;
        
        Cette fonction permet de filtrer un dataframe issu d'un tissu tumoral. 
    Dans un premier temps on supprime tous ce qui a été supprimé dans le dataframe normal complémentaire.
    Ensuite on supprime les varaiants dont le 'FILTER' est contenu dans la liste 'reject'
    
    Return :
    
        df_t : dataframe tumoral filtré.
    """
    
    df_t = copy.deepcopy(df_tumor) #deep copy pour ne pas ecraser l'original
    
    for chrom_pos in index_normal: #chrom_pos : tuple (#CHROM, #POS)
        chrom = chrom_pos[0] #CHROM
        pos = chrom_pos[1] #POS
        if(pos in df_tumor["POS"].loc[df_tumor["CHROM"] == chrom].values): #on regarde si la position indexee de df_normal est aussi dans df_tumor
            tmp_index = int(np.argwhere(df_tumor["POS"].loc[df_tumor["CHROM"] == chrom].values == pos)) #indexe de cette position
            #print(chrom, pos, tmp_index)
            df_t = df_t.drop(labels = tmp_index, axis=0) #on supprime la ligne ou il y a cette position
              
        df_t.index = range(0, len(df_t), 1) #reajustement des indexes apres le drop
    
    for muta in df_t.index: #idem que dans quality_filter_normal, on enleve les filters de mauvaise qualite
        if df_t["FILTER"][muta] in reject:
            df_t = df_t.drop(labels = muta, axis=0)
            
    df_t.index = range(0, len(df_t), 1)  #reajustement des indexes apres le drop
    
    return (df_t)

In [180]:
sample1_normal_chr1['CHROM'][661]

'1'

In [186]:
np.argwhere(sample1_normal_chr12["POS"].loc[sample1_tumor_chr12["CHROM"] == '2'].values == 15564799)

array([], shape=(0, 1), dtype=int64)

In [10]:
def global_filter(df_tumor, df_normal, reject = ['LowQual', 'INDEL_SPECIFIC_FILTERS;LowQual']):
    
    """
    Arguments :
    
        df_tumor : dataframe issu d'un vcf de tissu tumoral ;
        
        df_normal : dataframe issu d'un vcf de tissu sain ;
        
        reject : liste des filtres à ne pas garder ;
        
        Fonction qui prend en entrée 2 fichiesr vcf complémentaires (tumoral et normal issus d'un même individu),
    et qui fait appel aux fonctions quality_filter_normal et quality_filter_tumor afin de les filtrer,
    selon les filtres retenus dans la liste reject.
    
    Return :
    
        filtered_df_tumor : dataframe tumoral filtré ;
        
        filtered_df_normal : dataframe normal fitré.
            
            
    """
    
    filtered_df_normal = quality_filter_normal(df_normal, reject, index=True)[0]
    indexes2remove = quality_filter_normal(df_normal, reject, index=True)[1]
    
    filtered_df_tumor = quality_filter_tumor(df_tumor, indexes2remove, reject)
    
    print("Filtrage des variants de qualité :", *reject, sep='\n')
    
    return (filtered_df_tumor,
           filtered_df_normal)

In [101]:
sample1_normal = read_vcf("./vcf_files/sample1/Sample1-PBMC_normal_dna.vcf", QC=False)
sample1_tumor = read_vcf("./vcf_files/sample1/Sample1_tumor_dna.vcf", QC=False)
sample1_somatic = read_vcf("./vcf_files/sample1/Sample1_somatic_dna.vcf", QC=False)

Succès : extension vcf détectée
Contrôle qualité du VCF : Non fait
Succès : extension vcf détectée
Contrôle qualité du VCF : Non fait
Succès : extension vcf détectée
Contrôle qualité du VCF : Non fait


In [50]:
print(len(sample1_normal), len(sample1_tumor), len(sample1_somatic))

161052 170921 251


## Creation de fichiers 'lite'  (chr1)

In [136]:
def select_chr(df, chrom):
    """
    Arguments :
    
        df : dataframe.
        
        chrom : liste, le ou les chromosomes à conserver.
        
        Cette fonction permet de ne conserver qu'un ou plusieurs chromosomes parmi l'ensemble du dataframe,
    afin de réduire le dataframe et ainsi reduire les temps de chargement pour la suite de ce pipeline
    (ex : calcul du TMB qui peut être très long selon la taille du fichier etc ..)
    
    Return :
    
        new_df : dataframe n'ayant conservé que l'information sur les chromosomes renseignés par l'utilisateur.
    """
    new_df = copy.deepcopy(df)
    
    if (len(chrom) == 1): #Garder un seul chromosome
        new_df = df.loc[df["CHROM"] == str(chrom[0])]
        new_df.index = range(0, len(new_df), 1)  #reajustement des indexes
        return(new_df)
    
    elif (len(chrom) >1): #Conserver plusieurs chromosomes
        chrom = sorted(chrom) # numero chromosome dans l'ordre croissant
        new_df = df.loc[df["CHROM"] == str(chrom[0])]
        for i in range(1, len(chrom), 1):
            tmp_df = df.loc[df["CHROM"] == str(chrom[i])]
            new_df = pd.concat([new_df, tmp_df])
            #print(len(tmp_df), len(new_df))
            
        new_df.index = range(0, len(new_df), 1)
        #print("Seuls les chromosomes suivants ont bien été conservés : ", *chrom, sep = "\n")
        return(new_df)

In [154]:
sample1_normal_chr12 = select_chr(sample1_normal, ['1', '2'])
sample1_tumor_chr12 = select_chr(sample1_tumor, ['1', '2'])
sample1_somatic_chr12 = select_chr(sample1_somatic, ['1', '2'])

In [151]:
sample1_normal_chr1 = select_chr(sample1_normal, [1])
sample1_tumor_chr1 = select_chr(sample1_tumor, [1])
sample1_somatic_chr1 = select_chr(sample1_somatic, [1])

In [149]:
sample1_normal_chr2 = select_chr(sample1_normal, [2])
sample1_tumor_chr2 = select_chr(sample1_tumor, [2])
sample1_somatic_chr2 = select_chr(sample1_somatic, [2])

In [139]:
print(len(sample1_normal_chr12), len(sample1_tumor_chr12), len(sample1_somatic_chr12))

26783 28390 46


## Test du QC des filtres :

In [191]:
# F is for FILTERED
F_sample1_tumor_chr12,  F_sample1_normal_chr12= global_filter(sample1_tumor_chr12, sample1_normal_chr12)

Filtrage des variants de qualité :
LowQual
INDEL_SPECIFIC_FILTERS;LowQual


In [152]:
# F is for FILTERED
F_sample1_tumor_chr1,  F_sample1_normal_chr1= global_filter(sample1_tumor_chr1, sample1_normal_chr1)

Filtrage des variants de qualité :
LowQual
INDEL_SPECIFIC_FILTERS;LowQual


In [153]:
# F is for FILTERED
F_sample1_tumor_chr2,  F_sample1_normal_chr2= global_filter(sample1_tumor_chr2, sample1_normal_chr2)

Filtrage des variants de qualité :
LowQual
INDEL_SPECIFIC_FILTERS;LowQual


In [13]:
print(Counter(sample1_normal_chr1["FILTER"]),"\n\n",
      Counter(F_sample1_normal_chr1["FILTER"]))

Counter({'PASS': 14021, 'LowQual': 861, 'VQSRTrancheSNP99.90to100.00': 261, 'INDEL_SPECIFIC_FILTERS;LowQual': 125, 'INDEL_SPECIFIC_FILTERS': 51}) 

 Counter({'PASS': 14021, 'VQSRTrancheSNP99.90to100.00': 261, 'INDEL_SPECIFIC_FILTERS': 51})


In [14]:
print(Counter(sample1_tumor_chr1["FILTER"]),"\n\n",
      Counter(F_sample1_tumor_chr1["FILTER"]))

Counter({'PASS': 15052, 'LowQual': 533, 'VQSRTrancheSNP99.90to100.00': 261, 'INDEL_SPECIFIC_FILTERS;LowQual': 173, 'INDEL_SPECIFIC_FILTERS': 149}) 

 Counter({'PASS': 14559, 'VQSRTrancheSNP99.90to100.00': 247, 'INDEL_SPECIFIC_FILTERS': 122})


## Test des inputs :

In [15]:
#Test avec un fichier où il manque une colonne :

test_colonne = read_vcf("vcf_files/sample1/vcf_incomplet_manque_1_colonne.vcf", QC=True)
print(test_colonne)

Succès : extension vcf détectée
Ce fichier vcf est incomplet
Il manque la colonne POS
Contrôle qualité du VCF : Ok
False


In [16]:
#Test avec un fichier où in y a des NaN sur une ligne :

test_ligne = read_vcf("vcf_files/sample1/vcf_ligne_incomplete.vcf")
test_ligne.tail(4)

Succès : extension vcf détectée
Suppression des NaN : Ok
Contrôle qualité du VCF : Ok


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,Sample1-PBMC_1
6,1,762601,7,T,C,221.84,PASS,AC=2;AF=1;AN=2;DB;DP=6;FS=0;MLEAC=2;MLEAF=1;MQ...,GT:GATK:AD:DP:GQ:PL,"1/1:1/1:0,6:6:18:250,18,0"
7,1,762632,8,T,A,57.74,PASS,AC=2;AF=1;AN=2;DB;DP=2;FS=0;MLEAC=2;MLEAF=1;MQ...,GT:GATK:AD:DP:GQ:PL,"1/1:1/1:0,2:2:6:85,6,0"
8,1,777122,9,A,T,100.03,PASS,AC=2;AF=1;AN=2;DB;DP=4;FS=0;MLEAC=2;MLEAF=1;MQ...,GT:GATK:AD:DP:GQ:PL,"1/1:1/1:0,4:4:12:128,12,0"
9,1,783304,10,T,C,324.78,PASS,AC=2;AF=1;AN=2;DB;DP=8;FS=0;MLEAC=2;MLEAF=1;MQ...,GT:GATK:AD:DP:GQ:PL,"1/1:1/1:0,8:8:24:353,24,0"


In [17]:
sum(test_ligne.isna().sum(axis=1)) #On a plus de NaN !

0

In [22]:
sample1_tumor.head()
#sample1_somatic.head()
#sample1_normal.head()

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,Sample1
0,1,752721,1,A,G,729.77,PASS,AC=2;AF=1;AN=2;DB;DP=21;FS=0;MLEAC=2;MLEAF=1;M...,GT:GATK:AD:DP:GQ:PL,"1/1:1/1:0,21:21:63:758,63,0"
1,1,752894,2,T,C,367.77,VQSRTrancheSNP99.90to100.00,AC=2;AF=1;AN=2;DB;DP=10;FS=0;MLEAC=2;MLEAF=1;M...,GT:GATK:AD:DP:GQ:PL,"1/1:1/1:0,10:10:30:396,30,0"
2,1,761957,3,A,AT,1252.73,PASS,AC=2;AF=1;AN=2;DB;DP=31;FS=0;MLEAC=2;MLEAF=1;M...,GT:GATK:AD:DP:GQ:PL,"1/1:1/1:0,31:31:96:1290,96,0"
3,1,762273,4,G,A,11593.8,PASS,AC=2;AF=1;AN=2;DB;DP=357;FS=0;MLEAC=2;MLEAF=1;...,GT:GATK:AD:DP:GQ:PL,"1/1:1/1:0,357:357:99:11622,1071,0"
4,1,762589,5,G,C,1841.77,PASS,AC=2;AF=1;AN=2;DB;DP=42;FS=0;MLEAC=2;MLEAF=1;M...,GT:GATK:AD:DP:GQ:PL,"1/1:1/1:0,42:42:99:1870,126,0"


## Différence entre tumor et normal :

In [25]:
def compare(df_tumor, df_normal):
    """
    Arguments :
    
        df_tumor : dataframe issu d'un vcf de tissu tumoral ;
        
        df_normal : dataframe issu d'un vcf de tissu sain ;
    
            On regarde si une mutation présente dans le fichier vcf tumoral est également présente 
        dans le fichier vcf sain (dit normal). Dans le cas échéant on ne compte pas cette mutation 
        comme étant une mutation propre à la tumeur, car elle est peut être localisée à d'autres 
        endroits dans l'organisme (simple SNP par exemple). 
        Dans le cas contraire on peut considèrer cette mutation comme étant somatique est propre à la tumeur.
        
    Return :
    
        indexes : liste d'ID de mutation présentes dans le dataframe tumoral et absente dans le normal.
    """
    
    CHROMS = np.unique(df_tumor['CHROM'].values)
    indexes = []
    
    for _chr_ in CHROMS:
        df_t = df_tumor.loc[df_tumor["CHROM"] == _chr_]
        df_n = df_normal.loc[df_normal["CHROM"] == _chr_]
    
        #df to np
        POS_tumor = df_t['POS'].values
        POS_normal = df_n['POS'].values

        for muta in df_tumor.index:
            NB_ALT_tumor = len(list(df_t['ALT'][muta]))
            START = int(POS_tumor[muta])
            END = START + len(list(df_t['ALT'][muta]))

            #On identifie une mutation par ses positions de début et de fin (start et end)
            if (START in list(POS_normal)): #même debut ?
                # !!une meme position peut sur des chr differents!! 
                index = int(np.argwhere(POS_normal == START))
                if (END == int(POS_normal[index]) + len(list(df_n['ALT'][index]) ) ): #même fin ?
                    pass # non somatique
                else:
                    indexes.append(df_t['ID'][muta])
            else:
                indexes.append(df_t['ID'][muta])
                
    return (indexes)

In [26]:
indexes = compare(F_sample1_tumor_chr1, F_sample1_normal_chr1)
len(indexes)

1223

In [27]:
def create_somatic(tumor_path, somatic_path, indexes):
    """
    Arguments:
    
        tumor_path : chemin vers le fichier tumoral vcf ;
        
        somatic_path : chemin vers le futur fichier somatic vcf de sortie;
        
        indexes : indexes : liste d'ID de mutation présentes dans le dataframe tumoral et absente dans le normal. 
        
        Cette fonction permet d'écrire dans un fichier toutes les mutations présentes dans le tissus tumoral,
    et absentes dans le tissu normal. Cette comparaison est faite avec la fonction compare. 
    
    Return :
        
        Rien (0)
        
    """
    
    headers = []
    lines = []

    with open(tumor_path, "r") as f_in:
        for line in f_in:
            if line.startswith('#'):
                headers.append(line)
            elif not line.startswith('#'):
                lines.append(line)

    with open(somatic_path, "w") as f_out:
        for i in range(len(headers)):
                f_out.write(headers[i])
        for j in range(len(lines)-1):
            if (lines[j].split()[2] in indexes):
                f_out.write(lines[j])
                
    return(0)

In [53]:
create_somatic("./vcf_files/sample1/Sample1_tumor_dna_chr1.vcf", 
               "./vcf_files/sample1/Sample1_pre_somatic_chr1.vcf", indexes)

0

## Calcul du TMB :

In [29]:
# Savoir si une mutation est synonyme ou pas :

def is_synonymous():
    pass

In [30]:
def TMB_without_somatic(df_tumor, df_normal, exome_length = 1):
    
    """
    Arguments :
    
        df_tumor : dataframe issu d'un vcf de tissu tumoral ;
        
        df_normal : dataframe issu d'un vcf de tissu sain ;
        
        exome_length : int, tailler de l'exome de référence pour calculer un taux.
         
         
    Return :
    
        TMB : float, Taux de mutation.
    """
    
    TMB=len(compare(df_tumor, df_normal))
    return(TMB/exome_length)

In [31]:
def TMB_with_somatic(df_somatic, exome_length = 1):
    """
    
    Arguments :
    
        df_somatic : dataframe d'un vcf somatique.
        
        exome_length : int, tailler de l'exome de référence pour calculer un taux.
        
        Ici on calcul notre TMB directement à partir d'un fichier vcf somatic,
    il y a donc moins d'étapes. Le calcul revient à prendre 
    la longueur du nombre de mutations somatiques totales.
    
    Return :
    
        TMB : float, Taux de mutation.
    """
    
    TMB = len(df_somatic['ALT'].values)
    return (TMB/exome_length)

In [32]:
## Bcp trop long à calculer... (pas fini après plus d'1h)

#TMB_without_somatic(sample1_tumor, sample1_normal)

In [54]:
TMB_without_somatic(F_sample1_tumor_chr1, F_sample1_normal_chr1)

1223.0

In [34]:
TMB_with_somatic(sample1_somatic_chr1)

23.0

### Test des librairies pyvcf et cyvcf :


In [35]:
#pip install cyvcf2
#pip install pyvcf
#pip install hgvs

In [36]:
import cyvcf2
import vcf

In [37]:
## Importation d'un VCF avec la librairie cyvcf2 :
## Si les colonnes INFO et FORMAT sont de la forme : AC=40;AF=0.1;AN=2;BQ=36;DP=413;FA=0.1;INDEL=0;MC=C>T

def read_cyvcf(vcf):

    #Général :

    CHROM = []
    POS = []
    REF = []
    ALT = []
    QUAL = []
    FILTER = []

    # Détails de la section INFO du VCF :

    AN = [] #Total number of alleles in called genotypes
    AC = [] #Allele count in genotypes, for each ALT allele, in the same order as listed
    AF = [] #Allele Frequency in primary data, for each ALT allele, in the same order as listed
    BQ = [] #RMS base quality
    SB = [] #Strand bias
    FA = [] #Overall fraction of reads supporting ALT
    MC = [] #Modification base changes at this position
    MT = [] #Modification types at this position
    NS = [] #Number of Samples With Data
    DP = [] #Total Depth across samples
    VT = [] #Variant type, can be SNP, INS or DEL
    SS = [] #Variant status relative to non-adjacent Normal,0=wildtype,1=germline,2=somatic,3=LOH,4=post-transcriptional modification,5=unknown
    ORIGIN = [] #Where the call originated from, the tumor DNA, RNA, or both
    SOMATIC = [] #Indicates if record is a somatic mutation
    INDEL = [] #Number of indels for all samples
    START = [] #Number of reads starting at this position across all samples
    STOP = [] #Number of reads stopping at this position across all samples



    for record in cyvcf2.VCF(vcf):
        CHROM.append(record.CHROM)
        POS.append(record.POS)
        REF.append(record.REF)
        ALT.append(record.ALT)
        QUAL.append(record.QUAL)
        FILTER.append(record.FILTER)

        #record.INFO est un objet de type cyvcf, pour extraire les données il faut utiliser .get()
        AN.append(record.INFO.get("AN"))
        AC.append(record.INFO.get("AC"))
        AF.append(record.INFO.get("AF"))
        BQ.append(record.INFO.get("BQ"))
        SB.append(record.INFO.get("SB"))
        FA.append(record.INFO.get("FA"))
        MC.append(record.INFO.get("MC"))
        MT.append(record.INFO.get("MT"))
        NS.append(record.INFO.get("NS"))
        DP.append(record.INFO.get("DP"))
        VT.append(record.INFO.get("VT"))
        SS.append(record.INFO.get("SS"))
        ORIGIN.append(record.INFO.get("ORIGIN"))
        SOMATIC.append(record.INFO.get("SOMATIC"))
        INDEL.append(record.INFO.get("INDEL"))
        START.append(record.INFO.get("START"))
        STOP.append(record.INFO.get("STOP"))

    df_VCF = pd.DataFrame(list(zip(CHROM, POS, REF, ALT, QUAL, FILTER)), 
                      columns=["CHROM", "POS", "REF", "ALT", "QUAL", "FILTER"])

    df_VCF[ ["AN", "AC", "AF", "BQ", "SB", "FA", "MC", 
         "MT", "NS", "DP", "VT", "SS", "ORIGIN", 
         "SOMATIC", "INDEL", "START", "STOP"]] = pd.DataFrame(list(zip (AN, AC, AF, BQ, SB, FA, MC,
                                                                        MT, NS, DP, VT, SS, ORIGIN, 
                                                                        SOMATIC, INDEL, START, STOP)))
    return (df_VCF)

In [38]:
#radia = read_cyvcf("./vcf_files/test_radia.vcf")
#radia_alt = radia['ALT'].values
#print(radia_alt[0:6])

In [39]:
#radia.head()

In [40]:
#basic_radia_TMB = len(radia_alt)
#print(basic_radia_TMB)

### Fichier mutect.vcf

In [41]:
# Avec la librairie pyvcf :

def read_pyvcf(file):
    reader = vcf.Reader(open(file))
    df = pd.DataFrame([vars(r) for r in reader])
    out = df.merge(pd.DataFrame(df.INFO.tolist()),
                   left_index=True, right_index=True)
    return out

In [42]:
#mutect = read_pyvcf("./vcf_files/test_mutect.vcf")
#mutect_alt = mutect['ALT'].values
#print(mutect_alt[:6])

In [43]:
#basic_mutect_TMB = len(mutect_alt)
#print(basic_mutect_TMB)

In [44]:
#mutect.head()

In [45]:
#muse = read_pyvcf("./vcf_files/test_muse.vcf")
#muse.head()

In [46]:
#merged = read_pyvcf("./vcf_files/test_merged.vcf")
#merged.head()

### Calcul du TMB en comparant un tissu sain et un tissu malade 

Comme on n'arrive pas à trouver une paire de jeu de données correctes, on va la fabriquer à partir du fichier mutect.vcf : on va enlever les 2/3 des lignes du fichier, et on considèrera que c'est notre tissu sain de référence. 

"In order to filter out germline variants, the ideal situation would be to
sequence a matched non-tumor sample from each patient."

#### Fonction simple

On considère ici que si la mutation est présente dans le tissu sain, c'est une mutation synonyme.

In [47]:
#mutect_tumor = read_pyvcf("./vcf_files/test_mutect.vcf")
#mutect_sain = read_pyvcf("./vcf_files/test_mutect_sain.vcf")

### Cas où l'on n'a pas de tissu sain

On doit comparer nos mutations aux mutations d'une base de donnée pour voir si ces mutations sont dues au polymorphisme ou non.


On utilise la libraireie HGVS de python pour comparer nos variations à des variations courantes, contenues dans les BDD : https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6282708/ 

Tuto ici : https://github.com/biocommons/hgvs


In [48]:
#import hgvs.dataproviders.uta